In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-25 02:00:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-02-25 02:00:14 (6.04 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
books_review_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep=r"\t", header=True, inferSchema=True)

# Show DataFrame
books_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [ ]:
books_reviews_df= books_review_df.select("review_id","customer_id","product_id", "product_parent", "review_date")
books_reviews_df = books_reviews_df.withColumn("review_date",to_date(col("review_date"),"yyyy-mm-dd"))
books_reviews_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-01-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-01-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-01-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-01-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-01-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-01-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-01-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-01-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-01-14|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124| 2005-01-14|
|R2UP6XSVYJBJ2H|   24209213|0976822105|     731931430| 2005-01-14|
|R21SYDQ70ILUC0|   52253037|1580085695|     586052746| 2005-01

In [ ]:
books_reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
books_reviews_df.count()

3105520

In [ ]:
books_products_df= books_review_df.select("product_id","product_title")
books_products_df= books_products_df.dropDuplicates()
books_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0029148510|Acts of War: Beha...|
|1844494896|Waiting For Kate ...|
|0451216954|Dark Lover (Black...|
|006056718X|Often Wrong, Neve...|
|0965852768|101 Spooktacular ...|
|141960435X|Flip Flop Spanish...|
|0471282502|Experimental Orga...|
|0756613647|Universe: The Def...|
|B0006YGCE8|Seismic risk asse...|
|207030602X|L Etranger (Folio...|
|1569711224|Aliens: Hive  (2n...|
|0756613744|Go Figure!: A Tot...|
|0891073825|The Cross: God's ...|
|1595140840|            Teach Me|
|1844491412|Led Zeppelin: The...|
|0060737980|Surrounded by Idi...|
|0486265250|Ancient Egyptian ...|
|0763627798|           Shoe Baby|
|097559950X|Natural Cures "Th...|
|0595362478| Accursed Inhabitant|
+----------+--------------------+
only showing top 20 rows



In [ ]:
books_products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
books_products_df.count()

779733

In [ ]:
books_customer_df= books_review_df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
books_customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28735044|             9|
|   15607794|             1|
|   15596972|             1|
|   32371281|             2|
|   12084021|             1|
|   14462635|             1|
|   41052840|             5|
|   52068296|           105|
|   51739623|             1|
|   14544320|             1|
|   50093726|           151|
|   52925615|             1|
|   12266315|             1|
|   44917096|             1|
|   52833886|             1|
|   51854558|            57|
|   12404586|             2|
|   49850307|             8|
|   23478170|             1|
|   51088740|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
books_customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
books_customer_df.count()

1502380

Postgres Setup

In [ ]:
# Configure settings for RDS to write the tables
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to customers table in RDS

books_customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS

books_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to review_id_table table in RDS

books_reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)